In [1]:
import tensorflow as tf
import numpy as np
from PIL import Image

In [ ]:
labels = {
0: "T-shirt/top",
1: "Trouser",
2: "Pullover",
3: "Dress",
4: "Coat",
5: "Sandal",
6: "Shirt",
7: "Sneaker",
8: "Bag",
9: "Ankle boot"
}

In [ ]:
(X_train, Y_train), (X_test, Y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
def get_train_test_cat(x_train: np.ndarray, x_test: np.ndarray, y_train: np.ndarray, y_test: np.ndarray, label=0) -> (np.ndarray, np.ndarray):
    x_train = x_train[y_train == label]
    x_test = x_test[y_test == label]
    return x_train, x_test

In [ ]:
# TODO: get train data
# 1
##########################################################
_, _ = get_train_test_cat(X_train, X_test, Y_train, Y_test, label=7) # Hint: het is wat er uit de functie komt
##########################################################

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# TODO: Init datascalar
# 2
############################################################
scaler_in = MinMaxScaler()
x_flat = ... # Hint: het is een vector, en de functie staat hieronder
data_scaler = ... # Hint: sklearn gebruikt altijd fit
#############################################################

def unroll(data: np.ndarray) -> np.ndarray:
    """
    Maakt van de data een vector (samples, -1)
    :param data:
    :return:
    """
    return np.reshape(data, (len(data), -1))

def add_dummy_dim(data: np.ndarray) -> np.ndarray:
    """
    Voegd een extra dummy dim to aan het eind
    :param data:
    :return:
    """
    return np.expand_dims(data, axis=-1)

def norm_data(train: np.ndarray, test: np.ndarray, shape: str ='flat') -> (np.ndarray, np.ndarray):
    """
    Transformeerd data voor de autoencoder
    flat: een vector
    dummy: een extra dim aan het eind voor non compressie versie (is heel traag, gebruik deze niet)
    None: Geeft de originele dims terug. Werkt prima, maar niet omdat het hoort. (Zie tensorflow docs voor meer informatie)
    :param train:
    :param test:
    :param shape:
    :return:
    Genormaliseerde (eventueel getransformeerde) data
    """
    unroll_train = unroll(train)
    unroll_test = unroll(test)
    _train = data_scaler.transform(unroll_train)
    _test = data_scaler.transform(unroll_test)
    if shape=='flat':
        return _train, _test
    if shape=='dummy':
        return add_dummy_dim(train), add_dummy_dim(test)
    else:
        return np.reshape(_train, np.shape(train)), np.reshape(_test, np.shape(test))

def reverse_data(data: np.ndarray) -> np.ndarray:
    """
    Geeft weer de orginele data terug. Is voornamelijk voor het plotten van de data.
    In de tussentijd werk je alleen met de genormaliseerde data
    :param data:
    :return:
    Inverse van de getransformeerde data
    """
    if len(np.shape(data)) == 3:
        flat = unroll(data)
        inv_roll = data_scaler.inverse_transform(flat)
        img = np.reshape(inv_roll, (len(data), np.shape(data)[1], np.shape(data)[2]))
    elif len(np.shape(data)) == 4:
        flat = np.squeeze(data)
        inv_roll = data_scaler.inverse_transform(flat)
        img = np.reshape(inv_roll, (len(data), np.shape(data)[1], np.shape(data)[2]))
    else:
        flat = data
        inv_roll = data_scaler.inverse_transform(flat)
        img = np.reshape(inv_roll, (len(data), int(np.sqrt(np.shape(data)[1])), int(np.sqrt(np.shape(data)[1]))))

    return img


In [ ]:
# TODO: insert train set
# 3
####################################################
_, _ = norm_data(..., ..., shape="flat") # Hint: het is weer de train test set
####################################################

In [ ]:
import tensorflow as tf
from tensorflow.python.keras.models import Model

class AutoEncoder(Model):
  def __init__(self, size_features):
    super(AutoEncoder, self).__init__()
    self.encoder = tf.keras.Sequential([
                  # TODO: Maak hier je encoder
                    # Hint: wat een encoder-decoder maakt is de vorm van het model. Deze moet een zandloper zijn. Het gaat werken met elke gare combi, maar dat is niet de bedoeling
                    # Hint: de activate functies tussen layers is wat je zelf leuk vind. Gebruik bij twijfel altijd relu
                    # Hint: tf.keras.layers.Dense(128, activation="relu") is een layer
                    # Hint: De layers moeten via een mapping van de ene naar de andere kunnen. Tensorflow snapt de meeste wel, maar een multiplicatie gaat altijd goed
                    # Hint: Bv: 16 -> 8
                    # Hint: Het is geen DNN, dus we hebben geen 50 layers nodig, kan natuurlijk wel, maar dan mag je ook een uur trainen haha
                    # 4a
                    #####################################################

                    #####################################################
              ])
    self.decoder = tf.keras.Sequential([
                  # TODO: maak hier je decoder
                    # Hint: De laatste layer moet een geleidelijke activatie (smooth) functie zijn. Kies de juiste
                    # Hint: laatste layer moet ook de juiste aantal features bevatten
                    # Hint: Er is een populaire phase function (beginnend met een s- eindigend op -oid) als laatste activatie function
                    # 4b
                    #####################################################

                    #####################################################
              ])
  def call(self, x, **kwargs):
    # TODO: Maak de call
    # 4c
    # Hint: het is letterlijk de naam van het model
    # Hint: Negeer de kwargs, dat is voor tensorflow zelf
    return ...

In [ ]:
# TODO: init het model met de hoeveelheid features van je train set
# 5
##############################################
model = AutoEncoder(...) # Hint: bij een unroll/flat is het aantal features gelijk aan de lengte van de vector (aka het aantal pixels)
#############################################

In [ ]:
# TODO: train het model
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, mode="min")
# 6
####################################################################################################
model.compile(optimizer='adam', loss=...) # Hint: het maakt niet uit
history = model.fit(..., ..., epochs=200, batch_size=120, # Hint: Wat erin gaat moet er ook weer uit
                    validation_data=(..., ...), # Hint: Je test set
####################################################################################################
                    shuffle=True,
                    callbacks=[early_stopping]
                   )

In [ ]:
# TODO: Tijd om je model te testen
# 7
###########################################
pred = ... # Hint: er zijn 2 manieren: encode-decode OF gebruik de build in functie van tensorflow die altijd volgt na .fit
###########################################

In [ ]:
# TODO: om weer bruikbare data te maken moeten we terug naar de echte waardes
# 8
################################################################################################
pred_images = ... # Je moet de data omdraaien. De functie is al gemaakt. Als het goed is gegaan moet dit zonder aanpassingen gaan
real_images = ... # Hint: Een set die je eerder gemaakt hebt
################################################################################################
# (extra hint in het geval iets niet werkt: Je moet misschien een squeeze uitvoeren)

In [ ]:
# Optioneel: plot je data om te kijken of het goed is gegaan
#################################################
Image.fromarray(np.uint8(...))
#################################################

In [ ]:
def get_losses(train: np.ndarray, test: np.ndarray, y_labels: dict) -> np.ndarray:
    """
    Berekend alle losses van alle labels in de fashion MNIST set
    :param train:
    :param test:
    :param y_labels:
    :return:
    Een dict met key: label en value: losses
    """
    losses = dict()
    for label, value in y_labels.items():
        x_train, x_test = get_train_test_cat(X_train, X_test, Y_train, Y_test, label=label)
        _, test = norm_data(x_train, x_test, shape="flat")
        losses[value] = tf.keras.losses.mse(model.predict(unroll(test)), unroll(test)).numpy()
    return losses

In [ ]:
# Get losses van alle labels
performance = get_losses(X_train, X_test, y_labels=labels)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# TODO: selecteer een threshold
# 9
###########################################################
threshold = ... # Hint: loss van de ...
############################################################
# Hint: wat de threshold is maakt eigenlijk niet uit. Het is standaard om +/-2stdev te gebruiken oftewel ongeveer 5%. Maar dit hoeft niet.

In [ ]:
def plot_detection(losses: np.ndarray, name:str, offset:float=0) -> None:
    """
    Plot een histogram van alle losses en de threshold
    :param losses:
    :param name:
    :param offset:
    :return:
    None (een print statement)
    """
    sns.histplot(losses, bins=50)
    _ = plt.axvline(threshold, color='r', linewidth=3, linestyle='dashed', label='{:0.3f}'.format(threshold))

    passed = tf.math.less(losses, threshold+offset)
    print(f"Detection rate {name}: {100-tf.math.count_nonzero(passed).numpy()/len(losses)*100:.2f}%")

In [ ]:
for key, values in performance.items():
    plot_detection(values, key, offset=0.01)